# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from time import sleep
import smtplib
import email.message
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Instala o ChromeDriver e configura o serviço
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()

# Adiciona a opção para não fechar o navegador automaticamente
options.add_experimental_option("detach", True)

# Inicializa o navegador com o serviço e as opções configuradas
nav = webdriver.Chrome(service=service, options=options)
tabela = pd.read_excel("buscas.xlsx")
display(tabela)

def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    for palavra in lista_termos_banidos:
        if palavra in nome:
            return True
    return False

def verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome):
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            return False
    return True

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # Tratamento de valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    # Entra na aba shopping
    nav.get("https://shopping.google.com/?nord=1")

    # Pesquisar pelo produto
    search_box = WebDriverWait(nav, 10).until(
        EC.presence_of_element_located((By.ID, 'REsRA'))
    )
    search_box.send_keys(produto, Keys.ENTER)

    # Pegar as informações do produto
    lista_resultados = WebDriverWait(nav, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'i0X6df'))
    )

    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

        # Analisando se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # Analisando se ele tem todos os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)

        # Selecionar somente os elementos que têm termos banidos = False e ao mesmo tempo têm termos produto = True
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".").replace("+impostos", "")
            preco = float(preco)

            # Se o preço está entre o preço mínimo e o preço máximo
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))

    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # Tratamento de valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    # Inicializando o Buscapé e pesquisando
    nav.get("https://www.buscape.com.br/")
    search_box = WebDriverWait(nav, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input'))
    )
    search_box.send_keys(produto, Keys.ENTER)

    # Pegando os resultados
    sleep(5)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element(By.TAG_NAME, 'h2').text.lower()
        link = resultado.get_attribute("href")

        # Analisando se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        # Analisando se ele tem todos os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)
        # Analisando se está entre o preço mínimo e o preço máximo
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".").replace("+impostos", "")
            preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome, preco, link))

    return lista_ofertas

# Lista de ofertas
tabela_ofertas = pd.DataFrame()

for linha in tabela.index:
    # Informações para pesquisa
    produto = tabela.loc[linha, "Nome"]
    termos_banidos = tabela.loc[linha, "Termos banidos"]
    preco_minimo = tabela.loc[linha, "Preço mínimo"]
    preco_maximo = tabela.loc[linha, "Preço máximo"]

    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

display(tabela_ofertas)

# Exportar para o excel
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

#transformando tabela em html no corpo de email:

def df_to_html(df):
  return df.to_html(index=False, justify='center', classes='table table-striped')

#coloque a TABELA DE OFERTAS FINAL na função abaixo:

tabela_html = df_to_html(tabela_ofertas)


corpo_email = f"""
Olá prezados,

Segue tabela dos preços!

{tabela_html}


"""

# Criando o objeto MIMEMultipart
msg = MIMEMultipart()

# Configura o corpo do email como HTML
msg.attach(MIMEText(corpo_email, 'html'))

# Configura o cabeçalho do email
msg['Subject'] = 'Tabela de preços'
msg['From'] = 'pandagamer207@gmail.com' # EMAIL DE QUEM ESTÁ ENVIANDO
msg['To'] = 'martinssg199@gmail.com' # QUEM IRA RECEBER O EMAIL
password = 'tuxawmuwnavligvs' # NECESSARIO AUTENTICAÇÃO DE SENHA POR APP NO GMAIL PARA GERAR ESSA SENHA

try:
  # Conecta-se ao servidor SMTP e envia o email
  s = smtplib.SMTP('smtp.gmail.com', 587)
  s.starttls()
  s.login(msg['From'], password)
  s.sendmail(msg['From'], msg['To'], msg.as_string())
  s.quit()
  print('Email enviado')
except Exception as e:
  print('Erro ao enviar email:', str(e))

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,1000,1700
1,rtx 2060,pc gamer,700,1500


,produto,preco,link
0,iphone 12 64gb verde outlet,1537.65,https://www.google.com/url?url=https://www.tro...
1,usado: iphone 12 64gb preto outlet - trocafone,1659.00,https://www.google.com/url?url=https://www.bus...
2,apple iphone 12 - 64gb 128gb 256gb - todas as ...,1411.48,https://www.google.com/url?url=https://www.eba...
3,apple iphone 12 - 64gb 128gb 256gb - todas as ...,1216.98,https://www.google.com/url?url=https://www.eba...
4,"iphone se apple (64gb) branco tela 4.7"" câmera...",1399.00,https://www.google.com/url?url=https://www.ipl...
0,smartphone apple iphone 12 usado 64gb câmera d...,1659.00,https://www.buscape.com.br/celular/smartphone-...
1,smartphone apple iphone 12 vermelho usado 64gb,1659.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de video mancer geforce rtx 2060 heimdal...,1359.99,https://www.google.com/url?url=https://www.pic...
1,placa de video nvidia geforce rtx 2060 6gb gdd...,1399.99,https://www.google.com/url?url=https://www.kab...
2,rtx 2060 zotac 6gb | usado,1200.00,https://www.google.com/url?url=https://www.enj...


Email enviado
